In [1]:
%matplotlib inline

import logging
from pathlib import Path
import tarfile
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import pandas_path  # Path style access for pandas
from tqdm import tqdm
from pytorch_lightning.core.saving import save_hparams_to_yaml, load_hparams_from_yaml

from PIL import Image

import torch                    
import torchvision

import pytorch_lightning as pl

# for the purposes of this post, we'll filter
# much of the lovely logging info from our LightningModule
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.WARNING)

import os
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, roc_curve, confusion_matrix


from HatefulMemesDataset import HatefulMemesDataset
from LanguageAndVisionConcat import LanguageAndVisionConcat
from HatefulMemesModel import HatefulMemesModel

In [2]:
data_dir = '/media/alex/Storage/datasets/MADE/3_graduation/parthplc/archive/data/'

output_path = "model-outputs"
os.makedirs(output_path, exist_ok = True)

config_yaml = output_path + '/hparams.yaml'

train_path = data_dir + 'train.jsonl'
dev_path = data_dir + 'dev.jsonl'

train_samples_frame = pd.read_json(train_path, lines=True)

In [ ]:
images = [
    Image.open(
        data_dir + train_samples_frame.loc[i, "img"]
    ).convert("RGB")
    for i in range(5)
]

for image in images:
    print(image.size)

In [ ]:
# define a callable image_transform with Compose
image_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(224, 224)),
        torchvision.transforms.ToTensor()
    ]
)

# convert the images and prepare for visualization.
tensor_img = torch.stack(
    [image_transform(image) for image in images]
)
grid = torchvision.utils.make_grid(tensor_img)

# plot
plt.rcParams["figure.figsize"] = (20, 5)
plt.axis('off')
_ = plt.imshow(grid.permute(1, 2, 0))

In [ ]:
hparams = {    
    # Required hparams
    "train_path": train_path,
    "dev_path": dev_path,
    "img_dir": data_dir,
    
     #Optional hparams
    "embedding_dim": 150,
    "language_feature_dim": 300,
    "vision_feature_dim": 300,
    "fusion_output_size": 256,  
    
    "output_path": output_path,
    "dev_limit": None,
    "lr": 0.00005,
    "min_epochs": 20,
    "max_epochs": -1,
    "n_gpu": 1,
    "batch_size": 4,
    # allows us to "simulate" having larger batches 
    "accumulate_grad_batches": 16,
    "early_stop_patience": 3,
}

save_hparams_to_yaml(config_yaml=config_yaml, hparams=hparams)

In [ ]:
hateful_memes_model = HatefulMemesModel(hparams=hparams)
hateful_memes_model.fit()

In [ ]:
assert False

In [3]:
loaded_hparams = load_hparams_from_yaml(config_yaml=config_yaml)
loaded_hparams

{'accumulate_grad_batches': 16,
 'batch_size': 4,
 'dev_limit': None,
 'dev_path': '/media/alex/Storage/datasets/MADE/3_graduation/parthplc/archive/data/dev.jsonl',
 'early_stop_patience': 3,
 'embedding_dim': 150,
 'fusion_output_size': 256,
 'img_dir': '/media/alex/Storage/datasets/MADE/3_graduation/parthplc/archive/data/',
 'language_feature_dim': 300,
 'lr': 5e-05,
 'max_epochs': -1,
 'min_epochs': 20,
 'n_gpu': 1,
 'output_path': 'model-outputs',
 'train_path': '/media/alex/Storage/datasets/MADE/3_graduation/parthplc/archive/data/train.jsonl',
 'vision_feature_dim': 300}

In [5]:
hateful_memes_model = HatefulMemesModel.load_from_checkpoint(
    #'/media/alex/Storage/coding/MADE-graduation-projects/hateful_memes/drivendata_hateful-memes-benchmark/model-outputs/lightning_logs/version_0/checkpoints/epoch=0-step=96.ckpt',
    '/media/alex/Storage/coding/MADE-graduation-projects/hateful_memes/drivendata_hateful-memes-benchmark/model-outputs_v1/lightning_logs/version_0/checkpoints/epoch=2-step=288.ckpt',
    hparams=loaded_hparams#'hparams.yaml'
    #checkpoints[0]
)

Read 0M words
Number of words:  2262
Number of labels: 0
Progress: 100.0% words/sec/thread:  118331 lr:  0.000000 avg.loss:  3.013168 ETA:   0h 0m 0s


In [6]:
submission = hateful_memes_model.make_submission_frame(dev_path)
submission.head()

100%|█████████████████████████████████████████████████████████████████████████████| 125/125 [00:30<00:00,  4.15it/s]


,proba,label
id,,
8291,0.033967,0
46971,0.394693,0
3745,0.016486,0
83745,0.986041,1
80243,0.983324,1


In [7]:
test_data = pd.read_json(dev_path, lines=True)
test_data.head()

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime


In [8]:
acc_score = accuracy_score(submission.label.values, test_data.label.values)
auc_score = roc_auc_score(submission.label.values, test_data.label.values)
    
print(acc_score, auc_score)

0.552 0.5531011045029737
